Goal is to translate data from Pixelav (which comes in the form of 8x32 array) into something the board can understand (string of bits 00, 01, 10, 11 that corresponds to different charge thresholds). Then want to map it into array1.
\
\
**questions:**
- what are the different arrays 0,1,2 / what are we planning on filling them with 
- how do you take a time stamp? i don't actually know what pixelav is putting out so i don't know where that comes from

pixelav: 0:3 y, 0:15 x (8x32 --> 256 pixels). then each pixel goes to 2 bits which classify its charge. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# pretend pixelav output, based on figure 5 from the testing document
# this will be replaced by an actual pixelav file, but currently there is a size mismatch (we want 8x32 for the chip and the pixelav .out files give
# 13x21)... this is the next thing to work out. 
ideal_big_row = 4
ideal_big_col = 16
empty_array = np.zeros((ideal_big_row, ideal_big_col))
final_size = np.zeros((ideal_big_row*2, ideal_big_col*2))
count = np.arange(0,ideal_big_col*ideal_big_row,1)
pixels = []

for pixel in range(len(count)):
    start = 0 + (pixel*4)
    mini_array = np.array([[start+2, start], [start+3, start+1]])
    pixels.append(mini_array)
mini_array_idx = 0
for row in range(ideal_big_row):
    for col in range(ideal_big_col):
        row_start = np.abs(6-(4 - 1 - row)*2)
        col_start = (16 - 1 - col)*2
        final_size[row_start:row_start+2, col_start:col_start+2] = pixels[mini_array_idx]
        mini_array_idx += 1

test_vals = np.random.uniform(low = 400, high = 14000, size=(ideal_big_row*2, ideal_big_col*2))
low = 400
thresh1 = 1600
thresh2 = 2400

In [3]:
# will do this with a real parquet file soon but first just doing one 13x21 array with a fake cluster
real_pixelav_row = 13
real_pixelav_col = 21

real_pixelav_out = np.zeros((real_pixelav_row, real_pixelav_col))

# making a fake cluster
cluster_yend = 7
cluster_ystart = 3
cluster_xstart = 10
cluster_xend = 20
real_test_vals = np.random.uniform(low = 400, high = 14000, size = ((cluster_yend-cluster_ystart)*(cluster_xend-cluster_xstart),))

index = 0
for row in range(real_pixelav_row):
    for col in range(real_pixelav_col):
        if cluster_ystart <= row < cluster_yend and cluster_xstart <= col < cluster_xend:
            if index < len(real_test_vals):
                real_pixelav_out[row, col] = real_test_vals[index]
                index += 1

**mapping the (fake) pixelav output onto the ideal size (13x21-->8x32).** there are a couple ways to do this. we could take all the non-zero cluster information and just stick it in the center of the 8x32 array, but we aren't sure yet how/if this translation would affect the ultimate results. for now, i'm going to just truncate the values from the top and bottom rows (2 from the top and 3 from the bottom) and pad the extra columns of the pixelav output with zeros so that it fits into the 8x32 framework. i will also work in some "overflow" in case we accidentally chop off a part of a cluster, where i will add any data from the "chopped" pixels to the first row that we keep

In [4]:
print(f"Desired array size: {np.shape(final_size)} \nCurrent pixelav output size: {np.shape(real_pixelav_out)}")

# chopping the rows to fit in 8 and ensuring overflow
chop_start = 1
chop_stop = 9
overflow_top = np.zeros(real_pixelav_col)
overflow_bottom = np.zeros(real_pixelav_col)
for col in range(real_pixelav_col):
    for row in range(chop_start):
        overflow_top[col] += real_pixelav_out[row, col]
    for row in range(chop_stop + 1, real_pixelav_row):
        overflow_bottom[col] += real_pixelav_out[row, col]

real_pixelav_out[chop_start, :] += overflow_top
real_pixelav_out[chop_stop, :] += overflow_bottom 
chop_pixelav = real_pixelav_out[chop_start:chop_stop,:]

# padding with extra zeros to give 32 columns
padding = np.zeros((np.shape(final_size)[0], np.shape(final_size)[1]-np.shape(chop_pixelav)[1]))
match_pixelav = np.concatenate((chop_pixelav, padding), axis=1)

print(f"Final pixelav data size: {np.shape(match_pixelav)}")

Desired array size: (8, 32) 
Current pixelav output size: (13, 21)
Final pixelav data size: (8, 32)


In [14]:
pixel_data = []
for row in reversed(range(8)):
    for column in range(32):
        charge = match_pixelav[row, column]
        if charge <= low: 
            label = "00"
        elif low < charge < thresh1:
            label = "01"
        elif thresh1 < charge < thresh2:
            label = "10"
        else:
            label = "11"
        pixel_data.append(label)
bit_string = ''.join(reversed(pixel_data))

In [15]:
ar1row = 256
ar1col = 16
array1 = np.zeros((ar1row,ar1col))
len_shiftreg = (67+1)*16+4
len_bits = len(bit_string)
len_dnn = (255-100)*16+4*3

# fake numbers for shift register and dnn parts of the array
shiftreg = [0]*len_shiftreg
dnn = [0]*len_dnn

array1_flat = array1.flatten()

i = 0
for row in range(ar1row):
    for col in range(ar1col):
        if i < len_shiftreg:
            array1[row,col] = shiftreg[i]
        elif len_shiftreg <= i < len_shiftreg + len_bits:
            array1[row, col] = bit_string[i - len_shiftreg]
        elif i >= len_shiftreg+len_bits:
            array1[row, col] = dnn[i - (len_shiftreg+len_bits)]
        i+=1